# Westeros Tutorial Part 3a- Adding fossil resources (i)

This tutorial shows how to add fossil fuel resources in the form of resource supply curves to a MESSAGEix scenario. This includes defining `resource_volume` and `resource_cost`. Multiple resource categories can be added for a single commodity like coal, allowing, for example, a differentiation between '*reserves*' and '*resources*', the quality, or location of the resources, i.e., above or below ground, or on- or off-shore.

## Difference between reserves and resources
‘*Reserves*’ are generally defined as being those quantities for which geological and engineering information indicate with reasonable certainty that they can be recovered in the future from known reservoirs under existing economic and operating conditions. ‘*Resources*’ are detected quantities that cannot be profitably recovered with current technology, but might be recoverable in the future, as well as those quantities that are geologically possible, but yet to be found. Definitions are based on Rogner et al. (2012). 

In this tutorial, we add two categories of coal resources. We assume a resource potential based on the coal requirements by the `coal_ppl` in the baseline scenario. We also show the use of multiple resource categories. 

Two further tutorials on adding fossil resources are available, which will elaborate on the modelling of fossil resources. Part (ii) will introduce the constraint `resource_remaining`, for specifying what share of these resources must be preserved over time.  In Part (iii), we will further add a coal extraction technology, linking the coal resources to the primary energy level, which can be used to model energy requirements of the extraction process.

In [ ]:
# Importing required software packages
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

In [ ]:
# Loading modeling platform
mp = ixmp.Platform()

## Making a clone of the existing scenario '*baseline*'

In [ ]:
model = 'Westeros Electrified'
base = message_ix.Scenario(mp, model=model, scenario='baseline')
# Cloning with a new scenario name 'fossil_resources'
scen = base.clone(model, 'fossil_resources', 'illustration of adding fossil resources', keep_solution=False)
scen.check_out()

## Resource potentials in MESSAGEix

Introducing fossil resources requires the following steps to be carried out:
1. Add level and commodity required for resources:
   - Defining a new level and commodity which accounts for the coal resources.
   - Specifying this level as `resource`


2. Add resource potentials and corresponding parameters, including:
   - `grade`
   - `resource_volume`
   - `resource_cost`
   - `historical_extraction`


3. Linking the existing `coal_ppl` technology to the relevant resource, by updating the `input` parameter.

### 1. Defining level, grade, and commodity
We define two grades *a and b* for coal resources. Later, we define different costs for these grades.

In [ ]:
# Adding required information via MESSAGEix sets
commodity= 'coal'
level = 'resource'
scen.add_set('commodity', commodity)
scen.add_set('level', level)
scen.add_set('level_resource', level)
scen.add_set('grade', ['a', 'b'])

### 2. Setting up parameters
Based on the coal use in the *baseline* scenario, we set a sufficient potential for coal resources to be used by `coal_ppl`. In real examples, the amount of resources can be limited, which will be one of the criteria for employing different technologies. We update parameter `historical_extraction`, though it is not used in this tutorial. We show this as it can be useful when there is a history of resource extraction.

In [ ]:
# Retrieving model period information
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

In [ ]:
# Information for each "grade" including, potential, costs, historical extraction:
potentials = {'a': [1500, 10, 280],
              'b': [3500, 52, 0]}

In [ ]:
# Adding required information via MESSAGEix sets
for grade in potentials.keys():

    # Adding resource potentials
    # The index sets for resource_volume are ['node', 'commodity', 'grade', 'value', 'unit']
    df = pd.DataFrame({'node': [country],
                       'commodity': commodity,
                       'grade': grade,
                       'value': potentials[grade][0],
                       'unit': 'GWa'})
    scen.add_par('resource_volume', df)
    
    # Adding resource costs
    # The index sets for resource_cost are ['node', 'commodity', 'grade', 'year', 'value', 'unit']
    df = pd.DataFrame({'node': country,
                       'commodity': commodity,
                       'grade': grade,
                       'year': model_horizon.values.tolist(),
                       'value': potentials[grade][1],
                       'unit': 'USD/kWa'})
    scen.add_par('resource_cost', df)
    
    # Adding historical extraction
    # The index sets for historical_extraction are ['node', 'commodity', 'grade', 'year', 'value', 'unit']
    df = pd.DataFrame({'node': country,
                       'commodity': commodity,
                       'grade': grade,
                       'year': [690],
                       'value':  potentials[grade][2],
                       'unit': 'GWa'})
    scen.add_par('historical_extraction', df)

### 3. Linking `coal_ppl` to resources through `input` parameter

In [ ]:
# A base dataframe for "input" parameter
base_input = {
    'node_loc': country,
    'year_vtg': vintage_years
    ,
    'year_act': act_years,
    'mode': 'standard',
    'node_origin': country,
    'commodity': 'electricity',
    'time': 'year',
    'time_origin': 'year',
}

df = make_df(base_input, technology='coal_ppl', commodity='coal', 
                   level='resource', value=1, unit='%')

scen.add_par('input', df)

### Commit and solve

In [ ]:
scen.commit(comment='added coal resources')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

# Plotting Results

In [ ]:
from message_ix.reporting import Reporter
from message_ix.util.tutorial import prepare_plots

# Scenario without coal resource potential
base_rep = Reporter.from_scenario(base)
prepare_plots(base_rep)

# Scenario with coal resource potentials added
rep = Reporter.from_scenario(scen)
prepare_plots(rep)

## Activity
***
When comparing the results of the original scenario without coal resource potentials ('*baseline*') to the results of our newly modified scenario ('*fossil_resources*'), the activity of the both electricity generation technologies is the same.

### Scenario: '*baseline*'

In [ ]:
plants =["coal_ppl", "wind_ppl"]
base_rep.set_filters(t=plants)
base_rep.get("plot activity")

### Scenario: '*fossil_resource*'

In [ ]:
rep.set_filters(t=plants)
rep.get("plot activity")

## Capacity
***
As is the case for activity, capacity also remains the same in both scenarios.

### Scenario: '*baseline*'

In [ ]:
base_rep.get("plot capacity")

### Scenario: '*fossil_resource*'

In [ ]:
rep.get("plot capacity")

## Prices
***
The impact of introducing coal resources and therewith associated costs, only becomes noticeable when looking at prices for `light`.  Especially in the latter two timeperiods the impact is noticable, where prices increase  from below 2cents/kWhr in the '*baseline*' scenario to ~2.5cents/kWhr.

### Scenario: '*baseline*'

In [ ]:
filters = dict(t=None, c=["light"])
base_rep.set_filters(**filters)
base_rep.get("plot prices")

### Scenario: '*fossil_resource*'

In [ ]:
rep.set_filters(**filters)
rep.get("plot prices")

## Extraction
***
In the first plot below, we can see the coal supply curve. In the figure below, depicting extraction over time, we can see that the model uses up the available potential of the cheaper `grade` *a* early on. The switch to the more expensive `grade` *b* as of 710, which is the reason forthe increase in the price of `light`.

### Scenario: '*fossil_resource*'

### Coal supply curve

In [ ]:
rep.set_filters()
rep.get("plot fossil supply curve")

### Coal extraction

In [ ]:
rep.set_filters()
rep.get("plot extraction")

In [ ]:
mp.close_db()